## 기본 셋팅

In [104]:
import math
import collections

import numpy as np
import sympy as sy
import matplotlib.pyplot as plt

from scipy.optimize import fsolve

from coppeliasim_zmqremoteapi_client import RemoteAPIClient

In [105]:
# Connect to the simulator
client = RemoteAPIClient()
sim = client.require("sim")

print("Connected to CoppeliaSim")

Connected to CoppeliaSim


In [106]:
# Get arm's joints
joints = []
for i in range(5):
    joints.append(sim.getObject(f"/youBotArmJoint{i}"))

# Gripper joint
joints.append(sim.getObject(f"/youBotGripperJoint2"))

points = []

# Car reference frame
points.append(sim.getObject(f"/youBot_ref"))
# Joint-0 위치
points.append(sim.getObject(f"/p0_ref"))
# End Effector 위치
points.append(sim.getObject(f"/pe_ref"))
# Target 위치
points.append(sim.getObject(f"/Target"))

# joint 제어 모드 변경
for joint in joints:
    sim.setObjectInt32Param(
        joint,
        sim.jointintparam_dynctrlmode,
        sim.jointdynctrl_position,
    )

In [107]:
# joint angle 조회
def read_joints(joints):
    js = []
    for joint in joints:
        j = sim.getJointPosition(joint)
        js.append(j)
    return js

print("Initial joint angles:", read_joints(joints))

Initial joint angles: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [108]:
# point position & orientation 조회
def read_points(points):
    ps = []
    for point in points:
        p = sim.getObjectPosition(point)
        o = sim.getObjectOrientation(point)
        ps.append(np.array(p + o))
    return ps

def print_points(ps):
    for i, p in enumerate(ps):
        print(f"Point {i+1}: Position = {p[:3]}, Orientation = {p[3:]}")

ps = read_points(points)
print("Initial point positions and orientations:")
print_points(ps)

Initial point positions and orientations:
Point 1: Position = [2.50000000e-02 9.99960000e-01 1.53971835e-11], Orientation = [ 3.71924713e-15 -1.44328993e-15 -1.66533454e-16]
Point 2: Position = [0.0249  1.16618 0.099  ], Orientation = [-5.55111512e-17 -1.11022302e-16  5.55111512e-17]
Point 3: Position = [0.0249  1.19918 0.74   ], Orientation = [-2.52598440e-16 -4.22626393e-16  4.79537226e-17]
Point 4: Position = [0.125 1.55  0.02 ], Orientation = [-0.  0. -0.]


In [109]:
# joint 제어
def control_joint(joints, thetas):
    for joint, j in zip(joints, thetas):
        sim.setJointTargetPosition(joint, j)

In [110]:
# gripper 제어
def control_gripper(gripper, state):
    position = sim.getJointPosition(gripper)
    position += 0.005 if state else -0.005
    sim.setJointTargetPosition(gripper, position)
    return position

In [174]:
# control joint velocity

def trace_arm(joints, target_thetas):
    js = read_joints(joints)
    diff_sum = 0
    thetas = []
    for i, target in enumerate(target_thetas):
        diff = js[i] - target
        diff_sum += diff
        thetas.append(js[i] - min(0.1, max(-0.05, diff)))
    control_joint(joints, thetas)
    return diff_sum

In [112]:
# forward kinematics
def fk(thetas, params):
    j1, j2, j3, j4 = thetas[:4]
    j0, pc = params[:2]
    # 월드 기준 자동차
    dc = pc[5]
    TWC = np.array([
        [np.cos(dc), -np.sin(dc), 0, pc[0]],
        [np.sin(dc),  np.cos(dc), 0, pc[1]],
        [         0,           0, 1, pc[2]],
        [         0,           0, 0,     1]
    ])

    # 자동차 -> joint-0
    TC0 = np.array([ # 좌표이동 및 y축을 기준으로 90도 회전
        [1, 0, 0, 0.0],
        [0, 1, 0, 0.166],
        [0, 0, 1, 0.099],
        [0, 0, 0, 1]
    ]) @ np.array([
        [np.cos(j0), -np.sin(j0), 0, 0],
        [np.sin(j0),  np.cos(j0), 0, 0],
        [         0,           0, 1, 0],
        [         0,           0, 0, 1]
    ])
    TW0 = TWC @ TC0

    # joint-0 -> joint-1
    ay1 = np.pi / 2
    T01 = np.array([ # 좌표이동 및 y축을 기준으로 90도 회전
        [ np.cos(ay1), 0, np.sin(ay1), 0.0],
        [           0, 1,           0, 0.033],
        [-np.sin(ay1), 0, np.cos(ay1), 0.147],
        [           0, 0,           0, 1]
    ]) @ np.array([ # z축을 기준으로 j1만큼 회전
        [np.cos(j1), -np.sin(j1), 0, 0],
        [np.sin(j1),  np.cos(j1), 0, 0],
        [         0,           0, 1, 0],
        [         0,           0, 0, 1]
    ])
    TW1 = TW0 @ T01

    # joint-1 -> joint-2
    T12 = np.array([ # 좌표이동, 회전 없음
        [1, 0, 0, -0.155],
        [0, 1, 0,  0.0],
        [0, 0, 1,  0.0],
        [0, 0, 0,  1]
    ]) @ np.array([ # z축을 기준으로 j2만큼 회전
        [np.cos(j2), -np.sin(j2), 0, 0],
        [np.sin(j2),  np.cos(j2), 0, 0],
        [         0,           0, 1, 0],
        [         0,           0, 0, 1]
    ])
    TW2 = TW1 @ T12

    # joint-2 -> joint-3
    T23 = np.array([ # 좌표이동, 회전 없음
        [1, 0, 0, -0.135],
        [0, 1, 0,  0.0],
        [0, 0, 1,  0.0],
        [0, 0, 0,  1]
    ]) @ np.array([ # z축을 기준으로 j3만큼 회전
        [np.cos(j3), -np.sin(j3), 0, 0],
        [np.sin(j3),  np.cos(j3), 0, 0],
        [         0,           0, 1, 0],
        [         0,           0, 0, 1]
    ])
    TW3 = TW2 @ T23

    # joint-3 -> joint-4
    ay4 = -np.pi / 2
    T34 = np.array([ # 좌표이동 및 y축을 기준으로 -90도 회전
        [ np.cos(ay4), 0, np.sin(ay4), -0.081],
        [           0, 1,           0,  0.0],
        [-np.sin(ay4), 0, np.cos(ay4),  0.0],
        [           0,  0,          0,  1]
    ]) @ np.array([ # z축을 기준으로 j4만큼 회전
        [np.cos(j4), -np.sin(j4), 0, 0],
        [np.sin(j4),  np.cos(j4), 0, 0],
        [         0,           0, 1, 0],
        [         0,           0, 0, 1]
    ])
    TW4 = TW3 @ T34

    pe_hat = TW4 @ np.array([ 0.0,   0.0,   0.123, 1])

    return pe_hat[:3]

In [113]:
# inverse kinematics
def ik(thetas, params):  
    pt = params[-1][:3]
    pe_hat = fk(thetas, params)
    # theta 범위 검증
    if thetas[0] < np.deg2rad(-90) or np.deg2rad(75) < thetas[0]:
        return 10, 0, 0, 0
    elif thetas[1] < np.deg2rad(-131.00) or np.deg2rad(131.00) < thetas[1]:
        return 10, 0, 0, 0
    elif thetas[2] < np.deg2rad(-102.00) or np.deg2rad(102.00) < thetas[2]:
        return 10, 0, 0, 0
    elif thetas[3] < np.deg2rad(-90.00) or np.deg2rad(90.00) < thetas[3]:
        return 10, 0, 0, 0
    return np.linalg.norm(pe_hat - pt), 0, 0, 0

In [114]:
def solve(js, ps):
    p0, pt = ps[1], ps[-1]
    diff = pt[:2] - p0[:2]
    angle = math.atan2(diff[1], diff[0])
    j0 = angle - p0[-1] - np.pi / 2
    target_thetas = fsolve(
        ik,
        [js[1], js[2], js[3], js[4]],
        [j0, ps[0], ps[-1]]
    )
    return np.concatenate((np.array([j0]), target_thetas))

In [115]:
n_steps = 400

# 시뮬레이션 실행
sim.setStepping(True)
sim.startSimulation()

# 각 스텝 실행
js = read_joints(joints)
ps = read_points(points)
target_thetas = solve(js, ps)
place_thetas = [np.pi, -np.pi / 6, -np.pi / 2.7, -np.pi / 3, 0]
base_thetas = [0, 0, 0, 0, 0]
stage = 0
gripper_positions = collections.deque(maxlen=100)
for i in range(n_steps):
    if stage == 0: # move to target
        diff_sum = trace_arm(joints, target_thetas)
        if abs(diff_sum) < 0.005:
            stage = 1
    elif stage == 1: # grip target
        position = control_gripper(joints[-1], True)
        gripper_positions.append(position)
        if len(gripper_positions) > 5 and np.abs(gripper_positions[-1] - gripper_positions[-5]) < 0.001:
            stage = 2
    elif stage == 2:  # pick the target
        target_thetas[0] = np.pi
        target_thetas[1] = -np.pi / 12
        target_thetas[2] = -np.pi / 6
        diff_sum = trace_arm(joints, target_thetas)
        if abs(diff_sum) < 0.005:
            stage = 3
    elif stage == 3: # move to place
        diff_sum = trace_arm(joints, place_thetas)
        if abs(diff_sum) < 0.005:
            stage = 4
    elif stage == 4: # place the target
        position = control_gripper(joints[-1], False)
        gripper_positions.append(position)
        if len(gripper_positions) > 5 and np.abs(gripper_positions[-1] - gripper_positions[-5]) < 0.001:
            stage = 5
    elif stage == 5: # move to base
        diff_sum = trace_arm(joints, base_thetas)
        if abs(diff_sum) < 0.005:
            stage = 6
    sim.step()

# 시뮬레이션 종료
sim.stopSimulation()

C:\Users\joy4m\AppData\Local\Temp\ipykernel_89688\3481707928.py:6: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  target_thetas = fsolve(


# Configuration Sampling

In [14]:
# Configuration space 

# Joint range
joint_ranges = [(-180, 180),   # Joint 0
                (-90, 75),     # Joint 1
                (-131, 131),   # Joint 2
                (-102, 102),   # Joint 3
                (-90, 90)]     # Joint 4

def configuration_sampling(n_sample, joint_ranges):
    samples = []
    for _ in range(n_sample):
        sample = [np.random.uniform(joint_range[0], joint_range[1]) for joint_range in joint_ranges]
        samples.append(sample)
    return samples

n_sample = 100
samples = configuration_sampling(n_sample, joint_ranges)

for i, sample in enumerate(samples, 1):
    print(f"Sample {i}: {sample}")

Sample 1: [8.752805365851458, 24.756972488589952, 47.251533248388995, -101.16299408441478, 65.49988334613033]
Sample 2: [-134.01855186338997, 31.86311963478765, 37.49547985336821, 10.20034485399816, 61.52002050131415]
Sample 3: [-57.251618136063584, 36.96374172584683, 29.901244211140295, 67.42694632990725, 6.273672357456775]
Sample 4: [-169.15081177075035, -18.75308248031729, 18.05631241437814, -36.13883127726828, -30.941369303967512]
Sample 5: [-96.49483271306592, -1.29537962588293, -47.98794347276572, -61.73717871484395, -86.6043910758517]
Sample 6: [-94.21104597855017, 43.127176713079535, 69.8268355563591, -68.54454715117018, -89.01068325932285]
Sample 7: [28.161099375742765, -30.422761556667247, -108.42614090770878, 39.350150049174914, -47.872822532692105]
Sample 8: [-26.005348368721542, -0.80726712070161, 31.282655826372604, 23.80665854545093, -28.96674062165898]
Sample 9: [-156.14177141248453, -80.63175580036122, -69.51231801967339, 58.30063627894529, -27.192015052109838]
Sample 

In [34]:
# sampling된 각도를 joint position으로 setting
def set_joint_positions(joint_angles):
    for i, angle in enumerate(joint_angles):
        sim.setJointPosition(joints[i], angle)

In [15]:
# Get obstacle position
obs = sim.getObject(f"/Obstacle")
obs_pos = sim.getObjectPosition(obs)
obs_radius = 0.2 + 0.06

print("Obstacle position:", obs_pos)

Obstacle position: [-0.7749999999999999, 1.300000000000001, 0.45000000000000023]


In [16]:
def fk_with_links(thetas, params):
    j1, j2, j3, j4 = thetas[:4]
    j0, pc = params[:2]
    dc = pc[5]

    # 월드 기준 자동차 위치 및 방향
    TWC = np.array([
        [np.cos(dc), -np.sin(dc), 0, pc[0]],
        [np.sin(dc),  np.cos(dc), 0, pc[1]],
        [         0,           0, 1, pc[2]],
        [         0,           0, 0,     1]
    ])

    # 자동차 -> joint-0 변환
    TC0 = np.array([
        [1, 0, 0, 0.0],
        [0, 1, 0, 0.166],
        [0, 0, 1, 0.099],
        [0, 0, 0, 1]
    ]) @ np.array([
        [np.cos(j0), -np.sin(j0), 0, 0],
        [np.sin(j0),  np.cos(j0), 0, 0],
        [         0,           0, 1, 0],
        [         0,           0, 0, 1]
    ])
    TW0 = TWC @ TC0
    p0_hat = TW0[:3, 3]  # joint-0 위치

    # joint-0 -> joint-1 변환
    ay1 = np.pi / 2
    T01 = np.array([
        [ np.cos(ay1), 0, np.sin(ay1), 0.0],
        [           0, 1,           0, 0.033],
        [-np.sin(ay1), 0, np.cos(ay1), 0.147],
        [           0, 0,           0, 1]
    ]) @ np.array([
        [np.cos(j1), -np.sin(j1), 0, 0],
        [np.sin(j1),  np.cos(j1), 0, 0],
        [         0,           0, 1, 0],
        [         0,           0, 0, 1]
    ])
    TW1 = TW0 @ T01
    p1_hat = TW1[:3, 3]  # joint-1 위치

    # joint-1 -> joint-2 변환
    T12 = np.array([
        [1, 0, 0, -0.155],
        [0, 1, 0,  0.0],
        [0, 0, 1,  0.0],
        [0, 0, 0,  1]
    ]) @ np.array([
        [np.cos(j2), -np.sin(j2), 0, 0],
        [np.sin(j2),  np.cos(j2), 0, 0],
        [         0,           0, 1, 0],
        [         0,           0, 0, 1]
    ])
    TW2 = TW1 @ T12
    p2_hat = TW2[:3, 3]  # joint-2 위치

    # joint-2 -> joint-3 변환
    T23 = np.array([
        [1, 0, 0, -0.135],
        [0, 1, 0,  0.0],
        [0, 0, 1,  0.0],
        [0, 0, 0,  1]
    ]) @ np.array([
        [np.cos(j3), -np.sin(j3), 0, 0],
        [np.sin(j3),  np.cos(j3), 0, 0],
        [         0,           0, 1, 0],
        [         0,           0, 0, 1]
    ])
    TW3 = TW2 @ T23
    p3_hat = TW3[:3, 3]  # joint-3 위치

    # joint-3 -> joint-4 변환
    ay4 = -np.pi / 2
    T34 = np.array([
        [ np.cos(ay4), 0, np.sin(ay4), -0.081],
        [           0, 1,           0,  0.0],
        [-np.sin(ay4), 0, np.cos(ay4),  0.0],
        [           0,  0,          0,  1]
    ]) @ np.array([
        [np.cos(j4), -np.sin(j4), 0, 0],
        [np.sin(j4),  np.cos(j4), 0, 0],
        [         0,           0, 1, 0],
        [         0,           0, 0, 1]
    ])
    TW4 = TW3 @ T34
    p4_hat = TW4[:3, 3]  # joint-4 위치

    # 엔드 이펙터 위치 계산
    pe_hat = TW4 @ np.array([0.0, 0.0, 0.123, 1])
    
    return [p0_hat, p1_hat, p2_hat, p3_hat, p4_hat, pe_hat[:3]]

In [17]:
# Colllision check

def check_workspace_collision(link_positions, obs_position, obs_radius):
    for pos in link_positions:
        dist = np.linalg.norm(pos - obs_position)
        if dist < obs_radius:
            return True  # 충돌 발생
    return False  # 충돌 없음

# 각 샘플링된 각도에 Work space 충돌 여부 확인
for i, sample in enumerate(samples):
    pc = read_points(points)[0]  # Car reference 위치
    
    # fk_with_links를 통해 Work space에서의 링크 위치 계산
    link_positions = fk_with_links(sample, (sample[0], pc))

    # Work space 에서의 충돌 여부 확인
    is_collision = check_workspace_collision(link_positions, obs_pos, obs_radius)


    if is_collision:
        print(f"Sample {i+1} is in collision in workspace")
    else:
        print(f"Sample {i+1} is collision-free in workspace")

Sample 1 is collision-free in workspace
Sample 2 is collision-free in workspace
Sample 3 is collision-free in workspace
Sample 4 is collision-free in workspace
Sample 5 is collision-free in workspace
Sample 6 is collision-free in workspace
Sample 7 is collision-free in workspace
Sample 8 is collision-free in workspace
Sample 9 is collision-free in workspace
Sample 10 is collision-free in workspace
Sample 11 is collision-free in workspace
Sample 12 is collision-free in workspace
Sample 13 is collision-free in workspace
Sample 14 is collision-free in workspace
Sample 15 is collision-free in workspace
Sample 16 is collision-free in workspace
Sample 17 is collision-free in workspace
Sample 18 is collision-free in workspace
Sample 19 is collision-free in workspace
Sample 20 is collision-free in workspace
Sample 21 is collision-free in workspace
Sample 22 is collision-free in workspace
Sample 23 is collision-free in workspace
Sample 24 is collision-free in workspace
Sample 25 is collision-fr

In [18]:
n_steps = 500

# Run simulation
sim.setStepping(True)
sim.startSimulation()

# 4. 시뮬레이션 루프: 샘플링된 각도 적용 및 FK 결과 비교
for i, sample in enumerate(samples):
    print(f"Applying Sample {i+1}")
    
    # 고정된 스텝 수만큼 trace_arm 반복하여 목표 각도로 이동
    fixed_steps = 30  # 각 샘플에 대해 이동할 스텝 수
    for _ in range(fixed_steps):
        trace_arm(joints, sample)
        sim.step()  # 한 스텝 진행
    
    # 현재 포인트 위치 읽기 (Car reference 위치 포함)
    ps = read_points(points)
    pc = ps[0]  # Car reference frame 위치 및 방향

    # Forward Kinematics 계산
    end_effector_pos = fk(sample, (sample[0], pc))
    
    # 시뮬레이션에서 End Effector 위치 읽기
    pe_actual = np.array(sim.getObjectPosition(points[2]))

    # error measure
    error = np.linalg.norm(pe_actual - end_effector_pos)
    print(f"Sample {i+1}: FK End Effector = {end_effector_pos}, Simulated End Effector = {pe_actual}, Error = {error}")

# 5. 시뮬레이션 종료
sim.stopSimulation()
print("Simulation finished.")


Applying Sample 1
Sample 1: FK End Effector = [0.32349552 1.11378868 0.17598434], Simulated End Effector = [0.03304839 1.08502314 0.64094258], Error = 0.5489746383341341
Applying Sample 2
Sample 2: FK End Effector = [0.29295417 1.10564352 0.55343958], Simulated End Effector = [0.01491079 1.29064153 0.65971746], Error = 0.350467363116794
Applying Sample 3
Sample 3: FK End Effector = [0.22702993 1.36279722 0.23340096], Simulated End Effector = [0.04640523 0.97950904 0.46430185], Error = 0.4825456812796922
Applying Sample 4
Sample 4: FK End Effector = [-0.06421972  1.05693656 -0.11836199], Simulated End Effector = [-0.05746974  1.32127727  0.53320078], Error = 0.7031753820880026
Applying Sample 5
Sample 5: FK End Effector = [0.09317798 1.03004088 0.08916821], Simulated End Effector = [0.05008514 1.13230234 0.66944277], Error = 0.5907901200020049
Applying Sample 6
Sample 6: FK End Effector = [-0.03572524  1.27910599  0.45493954], Simulated End Effector = [0.06627651 1.17560555 0.58801348],

# Test


In [166]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
import networkx as nx

# Configuration space 설정 (각 관절의 범위)
joint_ranges = [(-180, 180),   # Joint 0
                (-90, 75),     # Joint 1
                (-131, 131),   # Joint 2
                (-102, 102),   # Joint 3
                (-90, 90)]     # Joint 4

In [190]:
# Get obstacle position
obs = sim.getObject(f"/Obstacle")
obs_pos = sim.getObjectPosition(obs)
obs_radius = 0.2 + 0.06

print("Obstacle position:", obs_pos)

Obstacle position: [0.19999999999999998, 1.4250000000000038, 0.45000000000000023]


In [168]:
# 샘플링 함수 정의
def configuration_sampling(n_sample, joint_ranges):
    samples = []
    for _ in range(n_sample):
        sample = [np.random.uniform(joint_range[0], joint_range[1]) for joint_range in joint_ranges]
        samples.append(sample)
    return np.array(samples)

In [169]:
# 충돌 체크 함수
def check_workspace_collision(link_positions, obs_position, obs_radius):
    for pos in link_positions:
        dist = np.linalg.norm(pos - obs_position)
        if dist < obs_radius:
            return True  # 충돌 발생
    return False  # 충돌 없음

In [211]:
# Configuration space 설정 (각 관절의 범위)
joint_ranges = [(-180, 180),   # Joint 0
                (-90, 75),     # Joint 1
                (-131, 131),   # Joint 2
                (-102, 102),   # Joint 3
                (-90, 90)]     # Joint 4

# 샘플링 함수 정의
def configuration_sampling(n_sample, joint_ranges):
    samples = []
    for _ in range(n_sample):
        sample = [np.random.uniform(joint_range[0], joint_range[1]) for joint_range in joint_ranges]
        samples.append(sample)
    return np.array(samples)

# 충돌 체크 함수
def check_workspace_collision(link_positions, obs_position, obs_radius):
    for pos in link_positions:
        dist = np.linalg.norm(pos - obs_position)
        if dist < obs_radius:
            return True  # 충돌 발생
    return False  # 충돌 없음

# 두 configuration 사이 충돌 체크
def is_path_collision_free(config1, config2, obs_position, obs_radius, num_steps=10):
    interpolated_configs = np.linspace(config1, config2, num=num_steps)
    for config in interpolated_configs:
        link_positions = fk_with_links(config, (config[0], read_points(points)[0]))  # 링크 위치 계산
        if check_workspace_collision(link_positions, obs_position, obs_radius):
            return False  # 경로 중간에 충돌 발생
    return True

# 샘플 생성
n_sample = 100
samples = configuration_sampling(n_sample, joint_ranges)

# 장애물 설정
obs = sim.getObject(f"/Obstacle")
obs_pos = sim.getObjectPosition(obs)  # 장애물 위치를 시뮬레이션에서 가져옴
obs_radius = 0.2 + 0.06  # 장애물 반지름 설정


# 충돌 없는 샘플만 수집
collision_free_samples = []
for sample in samples:
    pc = read_points(points)[0]  # Car reference 위치 (위치 데이터 예시)
    link_positions = fk_with_links(sample, (sample[0], pc))  # 링크 위치 계산
    is_collision = check_workspace_collision(link_positions, obs_pos, obs_radius)
    
    if not is_collision:
        collision_free_samples.append(sample)

collision_free_samples = np.array(collision_free_samples)
print(f"총 충돌 없는 노드 수: {len(collision_free_samples)}")

# KNN 생성
k = 5
knn = NearestNeighbors(n_neighbors=k, metric='euclidean')
knn.fit(collision_free_samples)

# 각 노드의 K-최근접 이웃 찾기
distances, indices = knn.kneighbors(collision_free_samples)

# 그래프 생성 및 충돌 없는 간선만 추가
graph = nx.Graph()
for i, sample in enumerate(collision_free_samples):
    graph.add_node(i, pos=sample)

for i, neighbors in enumerate(indices):
    for neighbor in neighbors:
        if i != neighbor:  # 자기 자신과의 연결 방지
            node1, node2 = collision_free_samples[i], collision_free_samples[neighbor]
            if is_path_collision_free(node1, node2, obs_pos, obs_radius):  # 충돌 없는 경우에만 간선 추가
                graph.add_edge(i, neighbor)

# Start와 Goal 지점 추가
base_thetas = [0, 0, 0, 0, 0]  # Start 지점
js = read_joints(joints)       # 관절 정보 예시
ps = read_points(points)       # 목표 위치 정보 예시
target_thetas = solve(js, ps)  # Goal 지점

# Start, Goal 지점의 인덱스
start_index = len(collision_free_samples)
goal_index = start_index + 1

# Start, Goal 충돌 확인 후 KNN 연결
for index, config in zip([start_index, goal_index], [base_thetas, target_thetas]):
    # Start/Goal 노드 추가
    graph.add_node(index, pos=config)

    # 충돌 없는 KNN 간선 추가
    distances, neighbors = knn.kneighbors([config])
    for neighbor in neighbors[0]:  # 이웃들에 대해
        if is_path_collision_free(config, collision_free_samples[neighbor], obs_pos, obs_radius):
            graph.add_edge(index, neighbor)

print(f"Start 노드 인덱스: {start_index}")
print(f"Goal 노드 인덱스: {goal_index}")
print(f"총 노드 개수 (Start와 Goal 포함): {len(graph.nodes)}")
print(f"총 간선 개수: {len(graph.edges)}")

if nx.has_path(graph, start_index, goal_index):
    path = nx.shortest_path(graph, start_index, goal_index)
    print(f"최단 경로: {path}")
else:
    print("경로가 존재하지 않습니다.")


총 충돌 없는 노드 수: 100


C:\Users\joy4m\AppData\Local\Temp\ipykernel_89688\3481707928.py:6: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  target_thetas = fsolve(


Start 노드 인덱스: 100
Goal 노드 인덱스: 101
총 노드 개수 (Start와 Goal 포함): 102
총 간선 개수: 279
최단 경로: [100, np.int64(36), 101]


In [212]:
for i in range(len(joints)):
    sim.setJointPosition(joints[i], 0)


In [215]:
import collections

# 시뮬레이션 설정
n_steps = 400
sim.setStepping(True)
sim.startSimulation()

path = nx.shortest_path(graph, start_index, goal_index)

# Start와 Goal을 향한 각도 값
js = read_joints(joints)
ps = read_points(points)
target_thetas = solve(js, ps)
place_thetas = [np.pi, -np.pi / 6, -np.pi / 2.7, -np.pi / 3, 0]
base_thetas = [0, 0, 0, 0, 0]

# 경로를 따라 로봇 팔 이동
stage = 0
gripper_positions = collections.deque(maxlen=100)


for i in range(n_steps):
    sim.step()  # 시뮬레이션의 각 스텝 실행
    
    # path1 따라 이동
    if stage == 0:  # move to first target
        for node in path:
            joint_angles = graph.nodes[node]['pos']
            # set_joint_positions(joint_angles)  # 각도 설정
            diff_sum = trace_arm(joints, joint_angles)  # 목표로 점진적 이동
            
            if abs(diff_sum) < 0.005:  # 목표에 거의 도달 시 다음 스테이지로
                stage = 1

    elif stage == 1:  # grip target
        position = control_gripper(joints[-1], True)  # 그립 조정
        gripper_positions.append(position)
        
        if len(gripper_positions) > 5 and np.abs(gripper_positions[-1] - gripper_positions[-5]) < 0.001:
            stage = 2  # 그립 완료, 다음 스테이지로

    elif stage == 2:  # pick the target
        target_thetas[0] = np.pi
        target_thetas[1] = -np.pi / 12
        target_thetas[2] = -np.pi / 6
        diff_sum = trace_arm(joints, target_thetas)
        if abs(diff_sum) < 0.005:
            stage = 3
    elif stage == 3: # move to place
        diff_sum = trace_arm(joints, place_thetas)
        if abs(diff_sum) < 0.005:
            stage = 4
    elif stage == 4: # place the target
        position = control_gripper(joints[-1], False)
        gripper_positions.append(position)
        if len(gripper_positions) > 5 and np.abs(gripper_positions[-1] - gripper_positions[-5]) < 0.001:
            stage = 5
    elif stage == 5: # move to base
        diff_sum = trace_arm(joints, base_thetas)
        if abs(diff_sum) < 0.005:
            stage = 6
    sim.step()
    

sim.stopSimulation()


C:\Users\joy4m\AppData\Local\Temp\ipykernel_89688\3481707928.py:6: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  target_thetas = fsolve(


KeyboardInterrupt: 